<a href="https://colab.research.google.com/github/Sadiagul914/AI201/blob/main/Langchain_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install -U langchain-google-genai

In [47]:
import google.generativeai as genai

In [48]:
!pip install langchain langchain_community

In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')



# Part1: Gemini Features like temperature, tokens, max retries etc

In [50]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    api_key=GOOGLE_API_KEY,
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2) # gemini features
llm.invoke("HelloWorld!")


AIMessage(content='Hello World!\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ddaf4dc3-0ae6-45e5-a3fb-7a2cd6949c45-0', usage_metadata={'input_tokens': 3, 'output_tokens': 4, 'total_tokens': 7, 'input_token_details': {'cache_read': 0}})

#Part2:More prompt templates

In [51]:
prompt_template = PromptTemplate(
    input_variables=["question"],
    template="You are a helpful assistant. Answer the following question:\n\n{question}"
)

# Create the LLM chain
chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain with a sample question
question = "What is LangChain?"
response = chain.run({"question": question})

print("Answer:", response)

Answer: LangChain is a framework designed to simplify the development of applications using large language models (LLMs). It provides a modular and extensible set of tools, components, and interfaces to connect LLMs with other sources of computation or data, allowing for more complex and useful applications.

Here's a breakdown of what that means and why it's important:

* **Connecting LLMs with other data:**  LLMs are powerful, but their knowledge is limited to their training data. LangChain lets you connect them to external data sources (like files, APIs, or databases) so they can access up-to-date information and personalize responses.

* **Enabling complex workflows:**  LangChain allows you to chain together multiple LLM calls or other operations (like making API calls or querying a database) to create sophisticated applications. For example, you could create a chatbot that accesses product information from a database, summarizes it using an LLM, and then uses another LLM to genera

In [37]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

#Part3: Adding Memory

In [39]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory



# Initialize memory
memory = ConversationBufferMemory()

# Create the conversation chain with memory
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory
)

# Simulate a multi-turn conversation
user_inputs = [
    "Hello, can you explain what LangChain is?",
    "How does memory work in LangChain?",
    "Can LangChain be used with Google Gemini for applications?"
]

for user_input in user_inputs:
    response = conversation_chain.run(user_input)
    print(f"User: {user_input}")
    print(f"Assistant: {response}\n")

# Display the stored memory for debugging
print("Conversation Memory:")
print(memory.buffer)

User: Hello, can you explain what LangChain is?
Assistant: Hello! LangChain is a framework specifically designed for developing applications powered by language models. Think of it as a toolbox filled with components that make it easier to build sophisticated applications that go beyond just simple chatbots.  It's built around the idea of "chains," which are sequences of calls to language models or other utilities. These chains can be quite complex, allowing for a wide range of functionalities.

A key feature of LangChain is its management of prompts.  It provides a structured way to create and manage prompts, including templating and versioning.  This is crucial because the quality of the output from a language model heavily depends on the quality of the prompt.  LangChain helps you keep track of your prompts and experiment with different variations systematically.

Furthermore, LangChain facilitates connecting language models to external data sources.  Imagine you want to build an ap

#Part4: Tool Calling

In [65]:
# Tool creation

# Tool binding

# Tool calling

from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b




print(multiply.name)
print(multiply.description)
print(multiply.args)
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)


multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 12.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b50c3c14-b29b-417f-8578-fcbb4143173b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '52f47ba9-7bdb-494e-b91b-624ab574a252', 'type': 'tool_call'}], usage_metadata={'input_tokens': 104, 'output_tokens': 3, 'total_tokens': 107, 'input_token_details': {'cache_read': 0}})

#Additional code to call weather API

In [52]:
from langchain_google_genai  import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory



# Step 2: Define tools
# Tool 1: Database query
def run_database_query(query):
    # Simulate querying a database
    return f"Database result for: {query}"

database_tool = Tool(
    name="DatabaseQuery",
    func=run_database_query,
    description="Executes database queries. Provide a query to fetch the result."
)

# Tool 2: Weather API
def get_weather(location):
    # Simulate fetching weather data
    return f"The weather in {location} is sunny and 75°F."

weather_tool = Tool(
    name="WeatherAPI",
    func=get_weather,
    description="Gets the current weather for a given location."
)

tools = [database_tool, weather_tool]

# Step 3: Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)



# Step 5: Simulate a multi-turn conversation
print("\n--- Starting Conversation ---\n")
user_inputs = [
    "What is the latest sales data from the database?",
    "Can you tell me the weather in New York City?"
]

for user_input in user_inputs:
    response = conversation_chain.run(user_input)
    print(f"User: {user_input}")
    print(f"Assistant: {response}\n")

# Step 6: Review conversation memory
print("\n--- Conversation Memory ---")
print(memory.load_memory_variables({}))


--- Starting Conversation ---

User: What is the latest sales data from the database?
Assistant: As I mentioned before, I do not have access to any external databases or private information, including sales data.  My responses are based on the information I was trained on, and I cannot directly interact with databases containing real-time or private data.  You would need to query the relevant database directly to retrieve that information.


User: Can you tell me the weather in New York City?
Assistant: I do not have access to real-time information, including current weather conditions.  To get the latest weather in New York City, I would suggest checking a dedicated weather app or website. These resources are typically connected to live meteorological data and can provide accurate and up-to-date forecasts.



--- Conversation Memory ---
{'chat_history': []}
